# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/SMSSpamCollection.csv",inferSchema=True,sep=',')

In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
class|text |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... |
ham |Ok lar... Joking wif u oni... |
spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's |
ham |U dun say so early hor... U c already then say... |
ham |Nah I don't think he goes to usf, he lives around here though |
spam |FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv |
ham |Even my brother is not like to speak with me. They treat me like aids patent. |
ham |As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune |
spam |WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only. |
spam |Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030 |
ham |I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today. |
spam |SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info |
spam |URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18 |
ham |I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.|
ham |I HAVE A DATE ON SUNDAY WITH WILL!! |
spam |XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL |
ham |Oh k...i'm watching here:) |
ham |Eh u remember how 2 spell his name... Yes i did. He v naughty make until i v wet. |
ham |Fine if thatÂ’s the way u feel. ThatÂ’s the way its gota b |
spam |England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/Ãº1.20 POBOXox36504W45WQ 16+ |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [6]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thatÂ’s t...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

## Clean and Prepare the Data

** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 148|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 158|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 156|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thatÂ’s t...| 58|
 spam|England v Macedon...| 156|
+-----+--------------------+------+
only showing top 20 rows

In [12]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+--------------------+------------------+
 class| avg(length)|
+--------------------+------------------+
 ham| 71.31626943005182|
 spam|138.88487282463186|
ham	HI BABE UAWAK...| 19.0|
 ham	Yeah| 38.0|
+--------------------+------------------+

## Feature Transformations

In [14]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [16]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [21]:
from pyspark.ml import Pipeline

In [22]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [23]:
cleaner = data_prep_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [26]:
clean_data = clean_data.select(['label','features'])

In [27]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13428,[7,10,31,6...|
 0.0|(13428,[0,24,296,...|
 1.0|(13428,[2,13,19,2...|
 0.0|(13428,[0,72,78,1...|
 0.0|(13428,[36,131,30...|
 1.0|(13428,[11,60,141...|
 0.0|(13428,[11,52,108...|
 0.0|(13428,[124,185,4...|
 1.0|(13428,[1,46,118,...|
 1.0|(13428,[0,1,13,26...|
 0.0|(13428,[18,42,119...|
 1.0|(13428,[8,17,35,8...|
 1.0|(13428,[13,28,46,...|
 0.0|(13428,[38,96,221...|
 0.0|(13428,[546,1684,...|
 1.0|(13428,[28,107,11...|
 0.0|(13428,[80,215,47...|
 0.0|(13428,[0,2,49,13...|
 0.0|(13428,[0,74,104,...|
 1.0|(13428,[4,28,33,5...|
+-----+--------------------+
only showing top 20 rows

In [28]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [29]:
spam_predictor = nb.fit(training)

In [30]:
data.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [31]:
test_results = spam_predictor.transform(testing)

In [32]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13428,[0,1,9,14,...|[-536.41869460574...|[1.0,1.2772763904...| 0.0|
 0.0|(13428,[0,1,12,33...|[-447.53575780613...|[1.0,3.3459674098...| 0.0|
 0.0|(13428,[0,1,14,81...|[-687.17358610260...|[1.0,3.0370843437...| 0.0|
 0.0|(13428,[0,1,18,20...|[-834.01093388781...|[1.0,6.9972068596...| 0.0|
 0.0|(13428,[0,1,23,63...|[-1312.5988633101...|[1.0,2.5455521011...| 0.0|
 0.0|(13428,[0,1,26,34...|[-1487.7275991323...|[0.99999993966667...| 0.0|
 0.0|(13428,[0,1,42,68...|[-616.33038611787...|[0.99998859817665...| 0.0|
 0.0|(13428,[0,1,69,10...|[-686.21843709101...|[1.0,1.4766646019...| 0.0|
 0.0|(13428,[0,1,149,1...|[-249.88810761005...|[0.78847969531024...| 0.0|
 0.0|(13428,[0,1,875,1...|[-96.917105520238...|[0.99999999042099...| 0.0|
 0.0|(13428,[0,2,3,5,2...|[-490.62836265645...|[1.0,1.2724004020...| 0.0|
 0.0|(13428,[0,2,4,8,1...|[-1325.8399102397...|[1.0,1.8051879768...| 0.0|
 0.0|(13428,[0,2,4,8,2...|[-1422.8298819870...|[1.0,1.2222890635...| 0.0|
 0.0|(13428,[0,2,4,8,2...|[-552.98104101320...|[1.0,3.2383421063...| 0.0|
 0.0|(13428,[0,2,4,10,...|[-1126.6586170705...|[1.0,2.5359719662...| 0.0|
 0.0|(13428,[0,2,4,39,...|[-1580.6907187872...|[0.99999994040358...| 0.0|
 0.0|(13428,[0,2,4,43,...|[-1905.0574561372...|[1.0,4.5982991952...| 0.0|
 0.0|(13428,[0,2,7,8,1...|[-453.51259398531...|[1.0,4.1988965933...| 0.0|
 0.0|(13428,[0,2,7,10,...|[-1330.3388201576...|[1.0,2.6089307677...| 0.0|
 0.0|(13428,[0,2,7,10,...|[-834.92494631865...|[1.0,1.6942417581...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9139678188021164